In [2]:
import pandas as pd
import datetime as dt

# 1. Load Data
try:
    df = pd.read_csv('cleaned_data.csv')
    df.columns = df.columns.str.strip()
    print("Data Loaded Successfully!")
except FileNotFoundError:
    print("Error: 'cleaned_data.csv' not found.")

# --- PART 1: CORE KPIs CALCULATION ---
total_sales = df['Sales'].sum()
total_orders = df['Order ID'].nunique()
aov = total_sales / total_orders
total_customers = df['Customer Name'].nunique()

print("\n--- CORE BUSINESS KPIs ---")
print(f"1. Total Sales:           ${total_sales:,.2f}")
print(f"2. Total Orders:          {total_orders}")
print(f"3. Avg Order Value (AOV): ${aov:.2f}")
print(f"4. Unique Customers:      {total_customers}")

# --- PART 2: DEEP-DIVE (CUSTOMER SEGMENTATION) ---
if 'Order Date' in df.columns:
    # Adding 'dayfirst=True' to fix the date format error
    df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
    
    # Check if any dates failed to convert
    if df['Order Date'].isnull().any():
        print("\nWarning: Some dates could not be converted. Trying mixed format...")
        df['Order Date'] = pd.to_datetime(df['Order Date'], format='mixed')

    latest_date = df['Order Date'].max() + dt.timedelta(days=1)

    # RFM Calculation
    rfm = df.groupby('Customer Name').agg({
        'Order Date': lambda x: (latest_date - x.max()).days,
        'Order ID': 'nunique',
        'Sales': 'sum'
    }).reset_index()

    rfm.columns = ['Customer Name', 'Recency', 'Frequency', 'Monetary']

    # Scoring (1 to 4)
    rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
    rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
    rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

    rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

    def segment_customer(score):
        if score == '444': return 'Best Customers'
        elif score[1] == '4': return 'Loyal Customers'
        elif score[2] == '4': return 'Big Spenders'
        elif score[0] == '1': return 'Lost Customers'
        else: return 'Standard Customers'

    rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)

    print("\n--- CUSTOMER SEGMENTATION SUMMARY ---")
    print(rfm['Segment'].value_counts())

    rfm.to_csv('Task3_RFM_Analysis.csv', index=False)
    print("\nSuccess! 'Task3_RFM_Analysis.csv' has been saved.")
else:
    print("\nError: 'Order Date' column not found.")

Data Loaded Successfully!

--- CORE BUSINESS KPIs ---
1. Total Sales:           $2,261,536.78
2. Total Orders:          4922
3. Avg Order Value (AOV): $459.48
4. Unique Customers:      793

--- CUSTOMER SEGMENTATION SUMMARY ---
Segment
Standard Customers    330
Loyal Customers       163
Lost Customers        158
Big Spenders          107
Best Customers         35
Name: count, dtype: int64

Success! 'Task3_RFM_Analysis.csv' has been saved.
